# Business Problem

Who are the stakeholders in this project? Who will be directly affected by the creation of this project?

A NHL team

What business problem(s) will this Data Science project solve for the organization?

An NHL team is looking to determine who is likely, or more likely to suffer a concussion based on past performance and information. 

What problems are inside the scope of this project?

What problems are outside the scope of this project?

What data sources are available to us?

Eliteprospect_scraper package and a csv of nhl injuries

What is the expected timeline for this project? Are there hard deadlines (e.g. "must be live before holiday season shopping") or is this an ongoing project?

Finish bones of project by 1/17/2023

Do stakeholders from different parts of the company or organization all have the exact same understanding about what this project is and isn't?

In [2]:
import pandas as pd
import numpy as np


In [10]:
df = pd.read_csv("df")
df.head()

,player,team,gp,g,a,tp,ppg,pim,+/-,link,season,league,playername,position,fw_def,Name,Games Missed,Concussion
0,Jaromír Jágr (RW),Pittsburgh Penguins,81,52,69,121,1.49,42,19,https://www.eliteprospects.com/player/8627/jar...,2000-01,nhl,Jaromír Jágr,RW,FW,0,0.0,0
1,Joe Sakic (C),Colorado Avalanche,82,54,64,118,1.44,30,45,https://www.eliteprospects.com/player/8862/joe...,2000-01,nhl,Joe Sakic,C,FW,0,0.0,0
2,Patrik Elias (LW),New Jersey Devils,82,40,56,96,1.17,51,45,https://www.eliteprospects.com/player/8698/pat...,2000-01,nhl,Patrik Elias,LW,FW,0,0.0,0
3,Alexei Kovalev (RW),Pittsburgh Penguins,79,44,51,95,1.2,96,12,https://www.eliteprospects.com/player/8670/ale...,2000-01,nhl,Alexei Kovalev,RW,FW,0,0.0,0
4,Jason Allison (C),Boston Bruins,82,36,59,95,1.16,85,-8,https://www.eliteprospects.com/player/9064/jas...,2000-01,nhl,Jason Allison,C,FW,0,0.0,0


In [12]:
df.columns

Index(['player', 'team', 'gp', 'g', 'a', 'tp', 'ppg', 'pim', '+/-', 'link',
       'season', 'league', 'playername', 'position', 'fw_def', 'Name',
       'Games Missed', 'Concussion'],
      dtype='object')

In [13]:
df.head()

,player,team,gp,g,a,tp,ppg,pim,+/-,link,season,league,playername,position,fw_def,Name,Games Missed,Concussion
0,Jaromír Jágr (RW),Pittsburgh Penguins,81,52,69,121,1.49,42,19,https://www.eliteprospects.com/player/8627/jar...,2000-01,nhl,Jaromír Jágr,RW,FW,0,0.0,0
1,Joe Sakic (C),Colorado Avalanche,82,54,64,118,1.44,30,45,https://www.eliteprospects.com/player/8862/joe...,2000-01,nhl,Joe Sakic,C,FW,0,0.0,0
2,Patrik Elias (LW),New Jersey Devils,82,40,56,96,1.17,51,45,https://www.eliteprospects.com/player/8698/pat...,2000-01,nhl,Patrik Elias,LW,FW,0,0.0,0
3,Alexei Kovalev (RW),Pittsburgh Penguins,79,44,51,95,1.2,96,12,https://www.eliteprospects.com/player/8670/ale...,2000-01,nhl,Alexei Kovalev,RW,FW,0,0.0,0
4,Jason Allison (C),Boston Bruins,82,36,59,95,1.16,85,-8,https://www.eliteprospects.com/player/9064/jas...,2000-01,nhl,Jason Allison,C,FW,0,0.0,0


In [24]:
df["season"].unique()


array(['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06',
       '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12',
       '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18',
       '2018-19', '2019-20', '2020-21'], dtype=object)

In [23]:
df.loc[(df['season'] == '2020-21') & (df['Concussion'] == 1)]

,player,team,gp,g,a,tp,ppg,pim,+/-,link,season,league,playername,position,fw_def,Name,Games Missed,Concussion
17797,Nathan MacKinnon (C/RW),Colorado Avalanche,48,20,45,65,1.35,37,22,https://www.eliteprospects.com/player/99204/na...,2020-21,nhl,Nathan MacKinnon,C/RW,FW,Nathan MacKinnon,4.0,1
17825,Pavel Buchnevich (RW/LW),New York Rangers,54,20,28,48,0.89,42,12,https://www.eliteprospects.com/player/158906/p...,2020-21,nhl,Pavel Buchnevich,RW/LW,FW,Pavel Buchnevich,3.0,1
17868,Andrew Copp (C/LW),Winnipeg Jets,55,15,24,39,0.71,20,2,https://www.eliteprospects.com/player/101962/a...,2020-21,nhl,Andrew Copp,C/LW,FW,Andrew Copp,6.0,1
17923,Neal Pionk (D),Winnipeg Jets,54,3,29,32,0.59,20,6,https://www.eliteprospects.com/player/196583/n...,2020-21,nhl,Neal Pionk,D,DEF,Neal Pionk,3.0,1
17928,Christian Dvorak (LW/C),Arizona Coyotes,56,17,14,31,0.55,12,-11,https://www.eliteprospects.com/player/194889/c...,2020-21,nhl,Christian Dvorak,LW/C,FW,Christian Dvorak,19.0,1
17968,Jake Muzzin (D),Toronto Maple Leafs,53,4,23,27,0.51,29,21,https://www.eliteprospects.com/player/13614/ja...,2020-21,nhl,Jake Muzzin,D,DEF,Jake Muzzin,7.0,1
17969,Jake Muzzin (D),Toronto Maple Leafs,53,4,23,27,0.51,29,21,https://www.eliteprospects.com/player/13614/ja...,2020-21,nhl,Jake Muzzin,D,DEF,Jake Muzzin,20.0,1
18024,Tyler Johnson (C),Tampa Bay Lightning,55,8,14,22,0.4,16,-1,https://www.eliteprospects.com/player/12078/ty...,2020-21,nhl,Tyler Johnson,C,FW,Tyler Johnson,6.0,1
18050,Scott Laughton (C/LW),Philadelphia Flyers,53,9,11,20,0.38,39,13,https://www.eliteprospects.com/player/76690/sc...,2020-21,nhl,Scott Laughton,C/LW,FW,Scott Laughton,12.0,1
18055,Nick Foligno (LW/RW),totals,49,7,13,20,0.41,32,2,https://www.eliteprospects.com/player/10675/ni...,2020-21,nhl,Nick Foligno,LW/RW,FW,Nick Foligno,4.0,1


In [25]:
injuries = pd.read_csv('nhl_injury.csv', encoding='latin')

In [26]:
injuries.head()

,ï»¿Season,Team,Position,Player,Injury Type,Cap Hit,Chip,Games Missed,Games Missed.1
0,2000/01,Anaheim,F,"Kariya, Paul",Foot,NaN,NaN,16,16
1,2000/01,Anaheim,F,"Leclerc, Mike",Abdominal,NaN,NaN,13,13
2,2000/01,Anaheim,F,"Leclerc, Mike",Knee,NaN,NaN,15,15
3,2000/01,Anaheim,F,"McDonald, Andy",Concussion,NaN,NaN,7,7
4,2000/01,Anaheim,F,"McInnis, Marty",Groin,NaN,NaN,3,3


In [28]:
injuries["ï»¿Season"].unique()

array(['2000/01', '2000/01 (playoffs)', '2001/02', '2001/02 (playoffs)',
       '2002/03', '2002/03 (playoffs)', '2003/04', '2003/04 (playoffs)',
       '2005/06', '2005/06 (playoffs)', '2006/07', '2006/07 (playoffs)',
       '2007/08', '2007/08 (playoffs)', '2008/09', '2008/09 (playoffs)',
       '2009/10', '2009/10 (playoffs)', '2010/11', '2010/11 (playoffs)',
       '2011/12', '2011/12 (playoffs)', '2012/13', '2012/13 (playoffs)',
       '2013/14', '2013/14 (playoffs)', '2014/15', '2014/15 (playoffs)',
       '2015/16', '2015/16 (playoffs)', '2016/17', '2016/17 (playoffs)',
       '2017/18', '2017/18 (playoffs)', '2018/19', '2018/19 (playoffs)',
       '2019/20', '2019/20 (playoffs+)', '2020/21', '2020/21 (playoffs)',
       '2021/22', '2021/22 (playoffs)', '2022/23'], dtype=object)

# Data Understanding

What data is available to us? Where does it live? Do we have the data, or can we scrape/buy/source the data from somewhere else?
Who controls the data sources, and what steps are needed to get access to the data?
What is our target?
What predictors are available to us?
What data types are the predictors we'll be working with?
What is the distribution of our data?
How many observations does our dataset contain? Do we have a lot of data? Only a little?
Do we have enough data to build a model? Will we need to use resampling methods?
How do we know the data is correct? How is the data collected? Is there a chance the data could be wrong?

# Data Preparation

Detecting and dealing with missing values
Data type conversions (e.g. numeric data mistakenly encoded as strings)
Checking for and removing multicollinearity (correlated predictors)
Normalizing our numeric data
Converting categorical data to numeric format through one-hot encodin

# Modeling 

Detecting and dealing with missing values
Data type conversions (e.g. numeric data mistakenly encoded as strings)
Checking for and removing multicollinearity (correlated predictors)
Normalizing our numeric data
Converting categorical data to numeric format through one-hot encodin

# Evaluation

During this step, we'll evaluate the results of our modeling efforts. Does our model solve the problems that we outlined all the way back during step 1? Why or why not?

# 1. Obtain

# 2. Scrub

# 3. Explore

# 4. Model

# 5. Interpret